## Instructions ##
971 yeast

linear regression, snp weight vector, 选top 10%， 20%， 30%。。

分别用这些snp来再跑linear regression的model( no gene)

再把这些snp放到gene上，再跑一次linear regression model

 

神经网络input snp

最多两千个feature，用linear regression来筛选

 

还要考虑promoter， enhancer（先不搞）

 

1. 先选snp （Science paper：看snp preproccesing的procedure）<5 删掉

2. fit linear regression model

3. 选大约2000个

4. 再用神经网络 （pytorch）



In [2]:
import pandas as pd 
import numpy as np
import bisect as bs
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import pandas as pd
import scipy.stats as stat


In [3]:
SNP_url = "./1011geneSNP.csv"
Gene_url = "./GCF_000146045.2_R64_genomic.gff"

snp_yeast_matrix = pd.read_csv(SNP_url, sep = ',' , error_bad_lines=False)
del snp_yeast_matrix['#CHROM']
del snp_yeast_matrix['POS']
del snp_yeast_matrix['REF']
del snp_yeast_matrix['ALT']
del snp_yeast_matrix['ANN[*].GENE']
del snp_yeast_matrix['ANN[*].GENEID']
index_list = snp_yeast_matrix["ID"].tolist()
del snp_yeast_matrix['ID']
snp_yeast_matrix = snp_yeast_matrix.transpose()
column_names = list(snp_yeast_matrix.index)
to_delete = ["ABC", "ABF", "ASP", "BGS","BGF", "BHL","BHQ","BID","BIR","BKG","BKN","CFH","SACE_YAL","SACE_YBA",
             "SACE_YBB","SACE_YBM","SACE_YBN","SACE_YBO","SACE_YBP","SACE_YCJ","SACE_YCS","SACE_YCT","SACE_YCU","SACE_YCV","SACE_YCW",
             "SACE_YCX","SACE_YCY","SACE_YCZ","SACE_YDA","SACE_YDB","SACE_YDC","SACE_YDD", "SACE_YDE","SACE_YDF","SACE_YDG","SACE_YDH","SACE_YDI",
             "SACE_YDJ", "SACE_YDK", "SACE_YDL"]
print(len(to_delete))
index = 0
for index in range(len(to_delete)):
    curr = to_delete[index]
    curr = curr+"_"+curr
    to_delete[index] = curr
    index+=1
to_delete_index = []
for item in to_delete:
    curr_index = column_names.index(item)
    to_delete_index.append(curr_index)
snp_yeast_matrix = snp_yeast_matrix.drop(to_delete)
snp_yeast_matrix.head()

40


,0,1,2,3,4,5,6,7,8,9,...,18921,18922,18923,18924,18925,18926,18927,18928,18929,18930
AAA_AAA,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
AAB_AAB,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
AAC_AAC,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAD_AAD,0,0,0,0,1,1,1,0,0,0,...,0,0,1,1,1,1,0,0,1,1
AAE_AAE,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
pheno_url = "./phenoMatrix_35ConditionsNormalizedByYPD.csv"
pheno_yeast_matrix = pd.read_csv(pheno_url, delimiter='\t', error_bad_lines=False)
YPD6AU = pheno_yeast_matrix["YPD6AU"]
YPDBENOMYL200 = pheno_yeast_matrix["YPDBENOMYL200"]
YPDNACL1M = pheno_yeast_matrix["YPDNACL1M"]
YPDSDS = pheno_yeast_matrix["YPDSDS"]
YPGALACTOSE = pheno_yeast_matrix["YPGALACTOSE"]
YPGLYCEROL = pheno_yeast_matrix["YPGLYCEROL"]
YPRIBOSE = pheno_yeast_matrix["YPRIBOSE"]
YPSORBITOL = pheno_yeast_matrix["YPSORBITOL"]
total_run = 100

In [5]:
sum = 0
weight_vector_1 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPD6AU, test_size=0.2)
    lr1 = linear_model.LinearRegression()
    lr1.fit(x_train, y_train)
    temp = np.array(lr1.coef_)
    weight_vector_1 = [x + y for x, y in zip(weight_vector_1, temp)]
    predictions_1 = lr1.predict(x_test)
    pcc_1 = stat.pearsonr(predictions_1, y_test)
    counter+=1
    sum += pcc_1[0]

avg_assoc_1 = sum/total_run
avg_weight_vector_1 = np.divide(weight_vector_1, total_run)
print(avg_assoc_1)
print(avg_weight_vector_1)
top2000_index = (-avg_weight_vector_1).argsort()[:2000]
print (top2000_index)
print (avg_weight_vector_1[top2000_index])

0.017625507644391997
[-2.40193329e+08 -9.51329668e+07 -6.91509560e+07 ... -2.07819641e+05
  1.72123812e+06 -3.65489209e+06]
[   22    40   177 ... 12580 13439  1064]
[35909712.86755651 30969496.06911329 30172606.27411069 ...
  2057249.79204882  2056066.05203741  2055314.69394521]


In [6]:
sum = 0
weight_vector_3 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDNACL1M, test_size=0.2)
    lr3 = linear_model.LinearRegression()
    lr3.fit(x_train, y_train)
    temp = np.array(lr3.coef_)
    weight_vector_3 = [x + y for x, y in zip(weight_vector_3, temp)]
    predictions_3 = lr3.predict(x_test)
    pcc_3 = stat.pearsonr(predictions_3, y_test)
    counter+=1
    sum += pcc_3[0]

avg_assoc_3 = sum/total_run
avg_weight_vector_3 = np.divide(weight_vector_3, total_run)
print(avg_assoc_3)
print(avg_weight_vector_3)
top2000_index_3 = (-avg_weight_vector_3).argsort()[:2000]
print (top2000_index_3)
print (avg_weight_vector_3[top2000_index_3])

0.037436911180733945
[-3.37849673e+05  6.46576234e+07 -1.76284387e+07 ...  5.74838207e+04
  6.64331609e+05 -1.09117706e+06]
[    1     4     7 ... 17459  8807   579]
[64657623.42498443 43433330.5155755  14566282.72546469 ...
   724000.37730427   723879.2780653    723841.16101986]


In [7]:
sum = 0
weight_vector_4 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDSDS, test_size=0.2)
    lr4 = linear_model.LinearRegression()
    lr4.fit(x_train, y_train)
    temp = np.array(lr4.coef_)
    weight_vector_4 = [x + y for x, y in zip(weight_vector_4, temp)]
    predictions_4 = lr4.predict(x_test)
    pcc_4 = stat.pearsonr(predictions_4, y_test)
    counter+=1
    sum += pcc_4[0]

avg_assoc_4 = sum/total_run
avg_weight_vector_4 = np.divide(weight_vector_4, total_run)
print(avg_assoc_4)
print(avg_weight_vector_4)
top2000_index_4 = (-avg_weight_vector_4).argsort()[:2000]
print (top2000_index_4)
print (avg_weight_vector_4[top2000_index_4])

0.050776698519380235
[ 1.80293179e+08  3.00320564e+07  1.00394208e+07 ... -3.23214271e+04
 -1.37441458e+06 -3.48155840e+05]
[   0    9    1 ... 8013 2623 8329]
[1.80293179e+08 3.77394678e+07 3.00320564e+07 ... 7.83034738e+05
 7.82003570e+05 7.81750319e+05]


In [8]:
sum = 0
weight_vector_2 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDBENOMYL200, test_size=0.2)
    lr2 = linear_model.LinearRegression()
    lr2.fit(x_train, y_train)
    temp = np.array(lr2.coef_)
    weight_vector_2 = [x + y for x, y in zip(weight_vector_2, temp)]
    predictions_2 = lr2.predict(x_test)
    pcc_2 = stat.pearsonr(predictions_2, y_test)
    counter+=1
    sum += pcc_2[0]

avg_assoc_2 = sum/total_run
avg_weight_vector_2 = np.divide(weight_vector_2, total_run)
print(avg_assoc_2)
print(avg_weight_vector_2)
top2000_index_2 = (-avg_weight_vector_2).argsort()[:2000]
print (top2000_index_2)
print (avg_weight_vector_2[top2000_index_2])

0.01487416324101254
[ -7086718.755928    19818053.69945217 -10674090.39858287 ...
    368086.6244495    -112899.05171859   -325806.63563745]
[    4     6     1 ... 15876 13231  7014]
[28971723.24723437 20022340.91732469 19818053.69945217 ...
   698925.65741911   698919.24679987   698725.96386096]


In [9]:
sum = 0
weight_vector_5 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPGALACTOSE, test_size=0.2)
    lr5 = linear_model.LinearRegression()
    lr5.fit(x_train, y_train)
    temp = np.array(lr5.coef_)
    weight_vector_5 = [x + y for x, y in zip(weight_vector_5, temp)]
    predictions_5 = lr5.predict(x_test)
    pcc_5 = stat.pearsonr(predictions_5, y_test)
    counter+=1
    sum += pcc_5[0]

avg_assoc_5 = sum/total_run
avg_weight_vector_5 = np.divide(weight_vector_5, total_run)
print(avg_assoc_5)
print(avg_weight_vector_5)
top2000_index_5 = (-avg_weight_vector_5).argsort()[:2000]
print (top2000_index_5)
print (avg_weight_vector_5[top2000_index_5])

0.027988724790353886
[-3.55220592e+08  1.09278129e+08 -2.61123680e+08 ... -4.55756319e+05
  2.54604957e+06 -5.72904128e+04]
[    1    11    10 ...  3537 12134 11862]
[1.09278129e+08 7.41107208e+07 6.94411198e+07 ... 1.69495390e+06
 1.69492387e+06 1.69119076e+06]


In [10]:
sum = 0
weight_vector_6 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPGLYCEROL, test_size=0.2)
    lr6 = linear_model.LinearRegression()
    lr6.fit(x_train, y_train)
    temp = np.array(lr6.coef_)
    weight_vector_6 = [x + y for x, y in zip(weight_vector_6, temp)]
    predictions_6 = lr6.predict(x_test)
    pcc_6 = stat.pearsonr(predictions_6, y_test)
    counter+=1
    sum += pcc_6[0]

avg_assoc_6 = sum/total_run
avg_weight_vector_6 = np.divide(weight_vector_6, total_run)
print(avg_assoc_6)
print(avg_weight_vector_6)
top2000_index_6 = (-avg_weight_vector_6).argsort()[:2000]
print (top2000_index_6)
print (avg_weight_vector_6[top2000_index_6])

0.01669042277109073
[ 9.57655442e+07  8.30312598e+07 -5.45238087e+07 ... -6.39761225e+03
 -4.70708416e+06  2.03677672e+06]
[    0     1     4 ... 13165 16181  7643]
[95765544.23048124 83031259.82149577 63159836.09703083 ...
  4184377.91238532  4183117.47498691  4180022.78792203]


In [11]:
sum = 0
weight_vector_7 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPRIBOSE, test_size=0.2)
    lr7 = linear_model.LinearRegression()
    lr7.fit(x_train, y_train)
    temp = np.array(lr7.coef_)
    weight_vector_7 = [x + y for x, y in zip(weight_vector_7, temp)]
    predictions_7 = lr7.predict(x_test)
    pcc_7 = stat.pearsonr(predictions_7, y_test)
    counter+=1
    sum += pcc_7[0]

avg_assoc_7 = sum/total_run
avg_weight_vector_7 = np.divide(weight_vector_7, total_run)
print(avg_assoc_7)
print(avg_weight_vector_7)
top2000_index_7 = (-avg_weight_vector_7).argsort()[:2000]
print (top2000_index_7)
print (avg_weight_vector_7[top2000_index_7])

0.012708775898420057
[ 1.37363610e+08  2.55347170e+07 -2.27010896e+07 ... -5.95999093e+05
 -1.03600312e+04  1.72148497e+06]
[    0     3    22 ...  9194  6502 15553]
[1.37363610e+08 3.90485854e+07 3.55737953e+07 ... 1.04004830e+06
 1.03954627e+06 1.03873070e+06]


In [12]:
sum = 0
weight_vector_8 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPSORBITOL, test_size=0.2)
    lr8 = linear_model.LinearRegression()
    lr8.fit(x_train, y_train)
    temp = np.array(lr8.coef_)
    weight_vector_8 = [x + y for x, y in zip(weight_vector_8, temp)]
    predictions_8 = lr8.predict(x_test)
    pcc_8 = stat.pearsonr(predictions_8, y_test)
    counter+=1
    sum += pcc_8[0]

avg_assoc_8 = sum/total_run
avg_weight_vector_8 = np.divide(weight_vector_8, total_run)
print(avg_assoc_8)
print(avg_weight_vector_8)
top2000_index_8 = (-avg_weight_vector_8).argsort()[:2000]
print (top2000_index_8)
print (avg_weight_vector_8[top2000_index_8])

0.00658958047780311
[-1.98248928e+08  2.49733290e+07 -2.84093445e+07 ...  6.25713982e+05
  3.72645493e+05 -1.22592640e+06]
[  14   18   19 ... 3716 2050 3460]
[56802836.23059125 42934110.51236925 37816279.76193699 ...
  2528406.61915241  2527849.51669522  2527272.76368826]


In [13]:
df_columns = ["YPD6AU", "YPDBENOMYL200", "YPDNACL1M", "YPDSDS", "YPGALACTOSE", "YPGLYCEROL", "YPRIBOSE", "YPSORBITOL"]
index_df = pd.DataFrame(list(zip(top2000_index, top2000_index_2, top2000_index_3, top2000_index_4, top2000_index_5, top2000_index_6, top2000_index_7, top2000_index_8)), columns = df_columns)


最后一层hiddeen nueron 数量为一

2000 1000 500 100 1 

adam optimize 要

batch normalization 

.cuda 挪到gpu

batch training 

batch

# Neural Network Class #

In [135]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as dt

class Net(nn.Module):
    def __init__(self, input_size, h1, h2, h3, output_size, lr):
        super(Net, self).__init__()
        self.l1 = nn.Linear(input_size, h1)
        self.l2 = nn.Linear(h1, h2)
        self.l3 = nn.Linear(h2, h3)
        self.l4 = nn.Linear(h3, output_size)
        self.relu = nn.ReLU()
        
        self.loss_fn = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr)

    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        out = self.relu(out)
        out = self.l4(out)
        return out
    
    #Not used. 
    def train_net(self, iterations, train_loader, test_loader):
        for epoch in range (num_epochs):
            total_loss = 0
            counter = 0
            for i, (data, target) in enumerate(train_loader):
                self.zero_grad()
                data = data.to(device)
                target = target.to(device)
                output = self.forward(data).to(device)
                loss = self.loss_fn(output, target)
                total_loss += loss
                counter += 1
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            check_correlation(test_loader, self)    
            epoch += 1
            print ('Epoch [{}/{}], Loss: {:.4f}' 
                .format(epoch, num_epochs, total_loss/len(train_loader)))
    
    def check_correlation(test_loader, net):
        counter = 0
        with torch.no_grad():
            total_err = 0.0
            for (data, target) in test_loader:
                data = data.to(device)
                target = target.cpu().numpy()
                output = torch.squeeze(net(data)).cpu().numpy()
                result = stat.pearsonr(target, output)
        print("Correlation: %f \n P-Value: %f" %(result[0], result[1]))

## Net Config Section ##

In [148]:
device = torch.device('cuda')
input_size = 2000
h1 = 1000
h2 = 200
h3 = 100
output_size = 1
lr = 0.0001
num_epochs = 10
batch_size = 200
split = 0.8


In [149]:
traning_data = torch.tensor(snp_yeast_matrix[top2000_index].values, dtype = torch.float)
answer = torch.tensor(YPD6AU, dtype = torch.float)
train_set = dt.TensorDataset(traning_data, answer)
train_size = int(split * len(train_set))
test_size = len(train_set) - train_size 

train_dataset, test_dataset = dt.random_split(train_set, [train_size, test_size])
train_loader = dt.DataLoader(dataset=train_dataset, batch_size=batch_size)

net = Net(input_size, h1, h2, h3, output_size, lr).to(device)

test_loader = dt.DataLoader(dataset=test_dataset, batch_size=batch_size)

net.train_net(num_epochs, train_loader, test_loader)

Correlation: -0.198183 
 P-Value: 0.005482
Epoch [1/10], Loss: 0.1250
Correlation: -0.192899 
 P-Value: 0.006897
Epoch [2/10], Loss: 0.0555
Correlation: -0.038615 
 P-Value: 0.591988
Epoch [3/10], Loss: 0.0380
Correlation: 0.266241 
 P-Value: 0.000169
Epoch [4/10], Loss: 0.0258
Correlation: 0.255406 
 P-Value: 0.000314
Epoch [5/10], Loss: 0.0266
Correlation: 0.084310 
 P-Value: 0.241261
Epoch [6/10], Loss: 0.0241
Correlation: 0.015851 
 P-Value: 0.825920
Epoch [7/10], Loss: 0.0218
Correlation: -0.151693 
 P-Value: 0.034266
Epoch [8/10], Loss: 0.0211
Correlation: -0.162698 
 P-Value: 0.023056
Epoch [9/10], Loss: 0.0205
Correlation: -0.057930 
 P-Value: 0.421151
Epoch [10/10], Loss: 0.0200


In [150]:
traning_data_2 = torch.tensor(snp_yeast_matrix[top2000_index_2].values, dtype = torch.float)
answer_2 = torch.tensor(YPDBENOMYL200, dtype = torch.float)
train_set_2 = dt.TensorDataset(traning_data_2, answer_2)

train_dataset_2, test_dataset_2 = dt.random_split(train_set_2, [train_size, test_size])
train_loader_2 = dt.DataLoader(dataset=train_dataset_2, batch_size=batch_size)
test_loader_2 = dt.DataLoader(dataset=test_dataset_2, batch_size=batch_size)

net_2 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_2.train_net(num_epochs, train_loader_2, test_loader_2)

Correlation: 0.105323 
 P-Value: 0.142817
Epoch [1/10], Loss: 0.0352
Correlation: 0.063321 
 P-Value: 0.379168
Epoch [2/10], Loss: 0.0153
Correlation: 0.011934 
 P-Value: 0.868487
Epoch [3/10], Loss: 0.0125
Correlation: 0.065269 
 P-Value: 0.364651
Epoch [4/10], Loss: 0.0113
Correlation: 0.084616 
 P-Value: 0.239553
Epoch [5/10], Loss: 0.0107
Correlation: 0.056291 
 P-Value: 0.434434
Epoch [6/10], Loss: 0.0103
Correlation: 0.015617 
 P-Value: 0.828452
Epoch [7/10], Loss: 0.0100
Correlation: 0.036896 
 P-Value: 0.608592
Epoch [8/10], Loss: 0.0100
Correlation: 0.016002 
 P-Value: 0.824288
Epoch [9/10], Loss: 0.0099
Correlation: -0.011238 
 P-Value: 0.876088
Epoch [10/10], Loss: 0.0098


In [151]:
traning_data_3 = torch.tensor(snp_yeast_matrix[top2000_index_3].values, dtype = torch.float)
answer_3 = torch.tensor(YPDNACL1M, dtype = torch.float)
train_set_3 = dt.TensorDataset(traning_data_3, answer_3)

train_dataset_3, test_dataset_3 = dt.random_split(train_set_3, [train_size, test_size])
train_loader_3 = dt.DataLoader(dataset=train_dataset_3, batch_size=batch_size)
test_loader_3 = dt.DataLoader(dataset=test_dataset_3, batch_size=batch_size)

net_3 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_3.train_net(num_epochs, train_loader_3, test_loader_3)

Correlation: -0.078553 
 P-Value: 0.275021
Epoch [1/10], Loss: 0.0630
Correlation: -0.008401 
 P-Value: 0.907211
Epoch [2/10], Loss: 0.0227
Correlation: 0.192797 
 P-Value: 0.006927
Epoch [3/10], Loss: 0.0160
Correlation: 0.207223 
 P-Value: 0.003652
Epoch [4/10], Loss: 0.0144
Correlation: 0.051339 
 P-Value: 0.475988
Epoch [5/10], Loss: 0.0125
Correlation: 0.001463 
 P-Value: 0.983804
Epoch [6/10], Loss: 0.0118
Correlation: 0.110147 
 P-Value: 0.125303
Epoch [7/10], Loss: 0.0110
Correlation: 0.117783 
 P-Value: 0.101029
Epoch [8/10], Loss: 0.0108
Correlation: 0.094068 
 P-Value: 0.190858
Epoch [9/10], Loss: 0.0108
Correlation: 0.088130 
 P-Value: 0.220524
Epoch [10/10], Loss: 0.0106


In [152]:
traning_data_4 = torch.tensor(snp_yeast_matrix[top2000_index_4].values, dtype = torch.float)
answer_4 = torch.tensor(YPDSDS, dtype = torch.float)
train_set_4 = dt.TensorDataset(traning_data_4, answer_4)

train_dataset_4, test_dataset_4 = dt.random_split(train_set_4, [train_size, test_size])
train_loader_4 = dt.DataLoader(dataset=train_dataset_4, batch_size=batch_size)
test_loader_4 = dt.DataLoader(dataset=test_dataset_4, batch_size=batch_size)

net_4 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_4.train_net(num_epochs, train_loader_4, test_loader_4)

Correlation: 0.135619 
 P-Value: 0.058707
Epoch [1/10], Loss: 0.0440
Correlation: -0.106545 
 P-Value: 0.138214
Epoch [2/10], Loss: 0.0292
Correlation: -0.153386 
 P-Value: 0.032288
Epoch [3/10], Loss: 0.0276
Correlation: 0.049778 
 P-Value: 0.489523
Epoch [4/10], Loss: 0.0269
Correlation: -0.025895 
 P-Value: 0.719340
Epoch [5/10], Loss: 0.0267
Correlation: -0.183215 
 P-Value: 0.010355
Epoch [6/10], Loss: 0.0264
Correlation: -0.170133 
 P-Value: 0.017413
Epoch [7/10], Loss: 0.0264
Correlation: -0.146172 
 P-Value: 0.041448
Epoch [8/10], Loss: 0.0263
Correlation: -0.097923 
 P-Value: 0.173229
Epoch [9/10], Loss: 0.0263
Correlation: -0.091464 
 P-Value: 0.203487
Epoch [10/10], Loss: 0.0263


In [153]:
traning_data_5 = torch.tensor(snp_yeast_matrix[top2000_index_5].values, dtype = torch.float)
answer_5 = torch.tensor(YPGALACTOSE, dtype = torch.float)
train_set_5 = dt.TensorDataset(traning_data_5, answer_5)

train_dataset_5, test_dataset_5 = dt.random_split(train_set_5, [train_size, test_size])
train_loader_5 = dt.DataLoader(dataset=train_dataset_5, batch_size=batch_size)
test_loader_5 = dt.DataLoader(dataset=test_dataset_5, batch_size=batch_size)

net_5 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_5.train_net(num_epochs, train_loader_5, test_loader_5)

Correlation: 0.149645 
 P-Value: 0.036797
Epoch [1/10], Loss: 0.6477
Correlation: 0.154437 
 P-Value: 0.031109
Epoch [2/10], Loss: 0.3905
Correlation: 0.143922 
 P-Value: 0.044718
Epoch [3/10], Loss: 0.2681
Correlation: 0.091020 
 P-Value: 0.205700
Epoch [4/10], Loss: 0.2069
Correlation: -0.056869 
 P-Value: 0.429721
Epoch [5/10], Loss: 0.1401
Correlation: -0.132319 
 P-Value: 0.065188
Epoch [6/10], Loss: 0.1305
Correlation: -0.125712 
 P-Value: 0.079923
Epoch [7/10], Loss: 0.1317
Correlation: -0.092125 
 P-Value: 0.200223
Epoch [8/10], Loss: 0.1210
Correlation: -0.077723 
 P-Value: 0.280144
Epoch [9/10], Loss: 0.1113
Correlation: -0.066405 
 P-Value: 0.356347
Epoch [10/10], Loss: 0.1077


In [154]:
traning_data_6 = torch.tensor(snp_yeast_matrix[top2000_index_6].values, dtype = torch.float)
answer_6 = torch.tensor(YPGLYCEROL, dtype = torch.float)
train_set_6 = dt.TensorDataset(traning_data_6, answer_6)

train_dataset_6, test_dataset_6 = dt.random_split(train_set_6, [train_size, test_size])
train_loader_6 = dt.DataLoader(dataset=train_dataset_6, batch_size=batch_size)
test_loader_6 = dt.DataLoader(dataset=test_dataset_6, batch_size=batch_size)

net_6 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_6.train_net(num_epochs, train_loader_6, test_loader_6)

Correlation: 0.065097 
 P-Value: 0.365921
Epoch [1/10], Loss: 0.2307
Correlation: 0.072467 
 P-Value: 0.314050
Epoch [2/10], Loss: 0.0942
Correlation: 0.112307 
 P-Value: 0.118019
Epoch [3/10], Loss: 0.0715
Correlation: 0.104831 
 P-Value: 0.144703
Epoch [4/10], Loss: 0.0538
Correlation: 0.082529 
 P-Value: 0.251379
Epoch [5/10], Loss: 0.0525
Correlation: 0.062075 
 P-Value: 0.388636
Epoch [6/10], Loss: 0.0492
Correlation: 0.023762 
 P-Value: 0.741606
Epoch [7/10], Loss: 0.0441
Correlation: -0.031889 
 P-Value: 0.658089
Epoch [8/10], Loss: 0.0428
Correlation: -0.099866 
 P-Value: 0.164814
Epoch [9/10], Loss: 0.0402
Correlation: -0.136849 
 P-Value: 0.056432
Epoch [10/10], Loss: 0.0394


In [155]:
traning_data_7 = torch.tensor(snp_yeast_matrix[top2000_index_7].values, dtype = torch.float)
answer_7 = torch.tensor(YPRIBOSE, dtype = torch.float)
train_set_7 = dt.TensorDataset(traning_data_7, answer_7)

train_dataset_7, test_dataset_7 = dt.random_split(train_set_7, [train_size, test_size])
train_loader_7 = dt.DataLoader(dataset=train_dataset_7, batch_size=batch_size)
test_loader_7 = dt.DataLoader(dataset=test_dataset_7, batch_size=batch_size)

net_7 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_7.train_net(num_epochs, train_loader_7, test_loader_7)

Correlation: 0.199129 
 P-Value: 0.005258
Epoch [1/10], Loss: 0.1729
Correlation: 0.196270 
 P-Value: 0.005961
Epoch [2/10], Loss: 0.0760
Correlation: 0.168728 
 P-Value: 0.018376
Epoch [3/10], Loss: 0.0505
Correlation: 0.036329 
 P-Value: 0.614109
Epoch [4/10], Loss: 0.0345
Correlation: -0.001790 
 P-Value: 0.980183
Epoch [5/10], Loss: 0.0334
Correlation: 0.067814 
 P-Value: 0.346211
Epoch [6/10], Loss: 0.0291
Correlation: 0.104366 
 P-Value: 0.146503
Epoch [7/10], Loss: 0.0259
Correlation: 0.021755 
 P-Value: 0.762749
Epoch [8/10], Loss: 0.0247
Correlation: -0.051956 
 P-Value: 0.470699
Epoch [9/10], Loss: 0.0233
Correlation: -0.042936 
 P-Value: 0.551185
Epoch [10/10], Loss: 0.0226


In [156]:
traning_data_8 = torch.tensor(snp_yeast_matrix[top2000_index_8].values, dtype = torch.float)
answer_8 = torch.tensor(YPSORBITOL, dtype = torch.float)
train_set_8 = dt.TensorDataset(traning_data_8, answer_8)

train_dataset_8, test_dataset_8 = dt.random_split(train_set_8, [train_size, test_size])
train_loader_8 = dt.DataLoader(dataset=train_dataset_8, batch_size=batch_size)
test_loader_8 = dt.DataLoader(dataset=test_dataset_8, batch_size=batch_size)

net_8 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_8.train_net(num_epochs, train_loader_8, test_loader_8)

Correlation: 0.154566 
 P-Value: 0.030966
Epoch [1/10], Loss: 0.0919
Correlation: 0.052128 
 P-Value: 0.469224
Epoch [2/10], Loss: 0.0418
Correlation: -0.151743 
 P-Value: 0.034206
Epoch [3/10], Loss: 0.0349
Correlation: -0.197579 
 P-Value: 0.005629
Epoch [4/10], Loss: 0.0308
Correlation: -0.136939 
 P-Value: 0.056268
Epoch [5/10], Loss: 0.0300
Correlation: -0.004157 
 P-Value: 0.954003
Epoch [6/10], Loss: 0.0280
Correlation: -0.061247 
 P-Value: 0.395007
Epoch [7/10], Loss: 0.0273
Correlation: -0.234795 
 P-Value: 0.000953
Epoch [8/10], Loss: 0.0262
Correlation: -0.237103 
 P-Value: 0.000846
Epoch [9/10], Loss: 0.0259
Correlation: -0.138230 
 P-Value: 0.053963
Epoch [10/10], Loss: 0.0257
